In [77]:
from  scapy.all import *
import netifaces
from subprocess import run

print ("Testing...") 

'''LINK STATUS'''
carrier=open('/sys/class/net/eth0/carrier','r') 
connect=carrier.readline()
if (connect=='0\n'):
    print ("operstate: down\n")
else:
    print ("operstate: up\n")
        
    '''IP state'''
    interface=open('/etc/network/interfaces','r')
    state=interface.readlines()
    static=False
    for i in state:
        if (0<i.find("static")):
            static=True
    if (static==True):        
        print("IP state: Static")
    else:
        print("IP state: DHCP")
            
    '''print network_inf'''
    try:
        addr=netifaces.ifaddresses('eth0')[2][0]['addr']
        netmask=netifaces.ifaddresses('eth0')[2][0]['netmask']
        gateway=netifaces.gateways()[2][0][0]
        network_inf = "Address: %s\nNetmask: %s\nGateway: %s\n"%(addr,netmask,gateway)
        print (network_inf)
    except:
        print ('no information about network\n')
        
    print ("Ping Gateway...")
    try:
        ping_gw='ping '+gateway+" -c 10"
        gw_test=subprocess.getoutput(ping_gw)
        ping_gw_result=(gw_test.split(','))[1]
        ping_gw_result=int(ping_gw_result.split(' ')[1])
    except:
        pass
    
    if (ping_gw_result>=9):
        print ("Sucess")
    else:
        print ("Failed")
print("Finished Test.")

Testing...
operstate: down

Finished Test.


In [2]:
from  scapy.all import *
catch =sniff (iface='eth0',filter ="udp port 1900 or arp" ,count=100)
catch

<Sniffed: TCP:0 UDP:36 ICMP:0 Other:64>

In [3]:
import ipaddress
lenth=len(catch)
data=[]
num=[]
for i in range(lenth):
    if (int(catch[i][0].type)==2048):
        if (catch[i][IP].src!="0.0.0.0"):
            tmp=catch[i][IP].src
            data.append(tmp)
            num.append(int(ipaddress.ip_address(tmp)))
    elif (int(catch[i][0].type)==2054):
        if (catch[i][ARP].psrc!="0.0.0.0"):
            tmp=catch[i][ARP].psrc
            data.append(tmp)
            num.append(int(ipaddress.ip_address(tmp)))

In [4]:
import pandas as pd
data={'number':num,'ip_address':data}
data=pd.DataFrame(data)

In [5]:
data3=data.drop_duplicates('ip_address',keep='first',inplace=False)
data3=data3.sort_values(by='number')

In [15]:
last=(data3.iloc[-1]).ip_address

In [16]:
first=(data3.iloc[0]).ip_address

In [17]:
lasto=int(ipaddress.IPv4Address(last))
firsto=int(ipaddress.IPv4Address(first))
mask=((lasto&firsto)|(~(lasto|firsto)))
tmp=int(ipaddress.IPv4Address("255.255.255.255"))+mask
tmp2=str(ipaddress.IPv4Address(tmp))
print (tmp2)

255.255.255.171


In [26]:
for i in range(2,31):
    temp=i
    temp=str(temp)
    s1="255.255.255.0/" + temp
    z=ipaddress.ip_network(s1,strict=False)
    print (z)
    mask=0
    if(ipaddress.IPv4Address(tmp2)<z.netmask):
        print (i-1)
        mask=i-1
        break

192.0.0.0/2
224.0.0.0/3
240.0.0.0/4
248.0.0.0/5
252.0.0.0/6
254.0.0.0/7
255.0.0.0/8
255.128.0.0/9
255.192.0.0/10
255.224.0.0/11
255.240.0.0/12
255.248.0.0/13
255.252.0.0/14
255.254.0.0/15
255.255.0.0/16
255.255.128.0/17
255.255.192.0/18
255.255.224.0/19
255.255.240.0/20
255.255.248.0/21
255.255.252.0/22
255.255.254.0/23
255.255.255.0/24
255.255.255.0/25
255.255.255.0/26
25


In [19]:
arp=[]
for i in range(lenth):
    if (int(catch[i][0].type)==2054):
        if (catch[i][ARP].psrc!="0.0.0.0"):
            src=catch[i][ARP].psrc
            dst=catch[i][ARP].pdst
            arp.append(src)
            arp.append(dst)
print (arp)

['192.168.1.51', '192.168.1.47', '192.168.1.51', '192.168.1.47', '192.168.1.51', '192.168.1.47', '192.168.1.1', '192.168.1.10', '192.168.1.1', '192.168.1.33', '192.168.1.1', '192.168.1.34', '192.168.1.42', '192.168.1.68', '192.168.1.42', '192.168.1.68', '192.168.1.42', '192.168.1.68', '192.168.1.1', '192.168.1.40', '192.168.1.1', '192.168.1.50', '192.168.1.1', '192.168.1.66', '192.168.1.1', '192.168.1.82', '192.168.1.42', '192.168.1.68', '192.168.1.42', '192.168.1.68', '192.168.1.42', '192.168.1.68', '192.168.1.42', '192.168.1.68', '192.168.1.1', '192.168.1.16', '192.168.1.1', '192.168.1.22', '192.168.1.1', '192.168.1.26', '192.168.1.1', '192.168.1.45', '192.168.1.1', '192.168.1.49', '192.168.1.1', '192.168.1.53', '192.168.1.1', '192.168.1.58', '192.168.1.1', '192.168.1.59', '192.168.1.15', '192.168.1.21', '192.168.1.15', '192.168.1.21', '192.168.1.15', '192.168.1.21', '192.168.1.15', '192.168.1.21', '192.168.1.51', '192.168.1.127', '192.168.1.51', '192.168.1.99', '192.168.1.23', '192.

In [20]:
dict={}
maxaddr=0
GW=''
for key in arp:
    dict[key]=dict.get(key,0)+1
    if (maxaddr<dict[key]):
        maxaddr=dict[key]
        GW=key
print(dict)

{'192.168.1.51': 17, '192.168.1.47': 3, '192.168.1.1': 33, '192.168.1.10': 2, '192.168.1.33': 2, '192.168.1.34': 2, '192.168.1.42': 7, '192.168.1.68': 7, '192.168.1.40': 1, '192.168.1.50': 1, '192.168.1.66': 1, '192.168.1.82': 1, '192.168.1.16': 1, '192.168.1.22': 1, '192.168.1.26': 1, '192.168.1.45': 1, '192.168.1.49': 1, '192.168.1.53': 1, '192.168.1.58': 1, '192.168.1.59': 1, '192.168.1.15': 7, '192.168.1.21': 7, '192.168.1.127': 7, '192.168.1.99': 7, '192.168.1.23': 2, '192.168.1.14': 1, '192.168.1.54': 1, '192.168.1.30': 1, '192.168.1.32': 1, '192.168.1.57': 1, '192.168.1.122': 1, '192.168.1.18': 1, '192.168.1.31': 1, '192.168.1.35': 1, '192.168.1.63': 1, '192.168.1.64': 1, '192.168.1.24': 1, '192.168.1.52': 1}


In [39]:
mask_tmp=ipaddress.ip_network("255.255.255.255/" + str(mask),strict=False)
mask_tmp.netmask

IPv4Address('255.255.255.128')

In [71]:
for i in range (0,len(data3)-1):
    ip=0
    #print ((data3.iloc[i]).number)
    if (((data3.iloc[i]).number-(data3.iloc[i+1]).number)<1):
        #print (i)
        ip=((data3.iloc[i]).number)+1
        #print(ip)
        break
ipaddress.ip_address(int (ip))

IPv4Address('192.168.1.2')

In [73]:
print ("Result")
print (ipaddress.ip_address(int (ip)))
print(mask_tmp.netmask)
print (GW)

Result
192.168.1.2
255.255.255.128
192.168.1.1


In [78]:
subprocess.getstatusoutput("ifconfig eth0 192.168.0.2 netmask 255.255.255.0")

(0, '')

In [79]:
    interface=open('/etc/network/interfaces','r')
    state=interface.readlines()
    static=False
    for i in state:
        if (0<i.find("static")):
            static=True
    if (static==True):        
        print("IP state: Static")
    else:
        print("IP state: DHCP")

IP state: DHCP
